In [5]:
import os
import uuid
import time
from flask import Flask, render_template, request, jsonify, session

# FORCE CPU (Windows-safe)
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["LLAMA_CPP_FORCE_CPU"] = "1"

from llama_cpp import Llama


In [6]:
print("🚀 Loading LLaMA model...")

MODEL_PATH = os.path.join(
    os.getcwd(),
    "model",
    "llama-3.2-3b-instruct.F16.gguf"
)

try:
    llm = Llama(
        model_path=MODEL_PATH,
        n_ctx=1024,
        n_threads=8,
        n_batch=64,
        verbose=True
    )
    print("✅ Model loaded successfully!")
except Exception as e:
    print("❌ Model failed to load:", e)
    llm = None


llama_model_loader: loaded meta data with 32 key-value pairs and 255 tensors from c:\Users\Thani\Documents\AI\model\llama-3.2-3b-instruct.F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Model_Nyota
llama_model_loader: - kv   3:                       general.quantized_by str              = Unsloth
llama_model_loader: - kv   4:                         general.size_label str              = 3.2B
llama_model_loader: - kv   5:                           general.repo_url str              = https://huggingface.co/unsloth
llama_model_loader: - kv   6:                               general.tags arr[str,2]       = ["unsloth", "lla

🚀 Loading LLaMA model...


init_tokenizer: initializing tokenizer for type 2
load: control token: 128098 '<|reserved_special_token_90|>' is not marked as EOG
load: control token: 128191 '<|reserved_special_token_183|>' is not marked as EOG
load: control token: 128130 '<|reserved_special_token_122|>' is not marked as EOG
load: control token: 128119 '<|reserved_special_token_111|>' is not marked as EOG
load: control token: 128136 '<|reserved_special_token_128|>' is not marked as EOG
load: control token: 128155 '<|reserved_special_token_147|>' is not marked as EOG
load: control token: 128196 '<|reserved_special_token_188|>' is not marked as EOG
load: control token: 128101 '<|reserved_special_token_93|>' is not marked as EOG
load: control token: 128138 '<|reserved_special_token_130|>' is not marked as EOG
load: control token: 128181 '<|reserved_special_token_173|>' is not marked as EOG
load: control token: 128034 '<|reserved_special_token_26|>' is not marked as EOG
load: control token: 128209 '<|reserved_special_tok

✅ Model loaded successfully!


CPU : LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'general.name': 'Model_Nyota', 'general.architecture': 'llama', 'general.type': 'model', 'llama.context_length': '131072', 'tokenizer.ggml.pre': 'llama-bpe', 'general.quantized_by': 'Unsloth', 'general.size_label': '3.2B', 'general.repo_url': 'https://huggingface.co/unsloth', 'llama.block_count': '28', 'llama.embedding_length': '3072', 'llama.feed_forward_length': '8192', 'llama.attention.head_count': '24', 'tokenizer.ggml.eos_token_id': '128009', 'general.file_type': '1', 'llama.attention.head_count_kv': '8', 'llama.rope.freq_base': '500000.000000', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.attention.key_length': '128', 'llama.attention.value_length': '128', 'llama.vocab_size': '128256', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_token_id': '128000', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'tokenizer.ggml.padding_token_id': '128004', 'tokenizer.ggml.add_bos_t

In [7]:
app = Flask(__name__)
app.secret_key = "dev-secret-key"

conversations = {}

@app.before_request
def before_request():
    if 'user_id' not in session:
        session['user_id'] = str(uuid.uuid4())
        conversations[session['user_id']] = []

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    if not llm:
        return jsonify(success=False, error="Model not loaded")

    user_input = request.json.get("message", "").strip()
    if not user_input:
        return jsonify(success=False, error="Empty message")

    result = llm(
        f"User: {user_input}\nAssistant:",
        max_tokens=256,
        temperature=0.7
    )

    response = result["choices"][0]["text"].strip()

    return jsonify(success=True, response=response)


In [8]:
app.run(
    host="192.168.1.50",
    port=5000,
    debug=False,
    use_reloader=False
)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.1.50:5000
Press CTRL+C to quit
192.168.1.50 - - [05/Jan/2026 11:17:07] code 400, message Bad request version ('\x00\x02\x01\x00ÿ\x01\x00\x01\x00\x00\x12\x00\x00\x00\x17\x00\x00\x00')
192.168.1.50 - - [05/Jan/2026 11:17:07] "\x16\x03\x01\x06à\x01\x00\x06Ü\x03\x03ß¤Ï\x97©\x8c$4CÃnbÚã\x8cÝ\x931)è\x05>¬\x14f*Ó¾'\x16\x9eÿ Õb\x8cõ\x8f\x07¬D² j¼¹\x04à\x84@Kp>\x91î¹\x16ò\x93\x1bþn\x8b8Æ\x00 **\x13\x03\x13\x01\x13\x02Ì©Ì¨À+À/À,À0À\x13À\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x06sªª\x00\x00\x00\x0b\x00\x02\x01\x00ÿ\x01\x00\x01\x00\x00\x12\x00\x00\x00\x17\x00\x00\x00" HTTPStatus.BAD_REQUEST -
192.168.1.50 - - [05/Jan/2026 11:17:07] code 400, message Bad request version ('\x00\x02\x01\x00\x00')
192.168.1.50 - - [05/Jan/2026 11:17:07] "\x16\x03\x01\x07\x00\x01\x00\x06ü\x03\x03\x8cG^\x93\x80\x97PÂ¬Éu\x00Ý¸\x9b(éþq\x0f<¾:Êk[\x10Î\x93L"ì \x0d\x90[wé\\Óÿ¶ Mj«©q×úî\x92\x88ër~®Ìí¾ÍR=Ðz\x00 zz\x13\x03\x13\x01\x13\x02Ì©Ì¨À+À/À,À0À\x13À\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x06\x9